In [ ]:
from dataclasses import dataclass
from typing import Union, Any

Level = Union[int, str]

class RichTypeError(TypeError):
    pass

@dataclass(frozen=True, slots=True)
class Arg:
    name: str

@dataclass(frozen=True, slots=True)
class Val:
    value: Any

def do_shit(level):
    raise RichTypeError(
        "Expected", Arg("level"), "to be", Level, "given", type(level),
        Val(level)
    )

In [33]:
from dataclasses import dataclass, replace, fields
from functools import cached_property
from inspect import signature
from threading import RLock

from splatlog.lib.collections import partition_mapping

class FmtOptsMeta(type):
    def __init__(cls, *args, **kwds):
        cls._field_names = None
        cls._cache_lock = RLock()

    @property
    def field_names(cls) -> set[str]:
        value = cls._field_names
        if value is not None:
            return value

        with cls._cache_lock:
            value = cls._field_names
            if value is not None:
                return value
            
            value = {field.name for field in fields(cls)}
            cls._field_names = value

            return value

@dataclass(frozen=True)
class FmtOpts(metaclass=FmtOptsMeta):
    def __new__(cls, *args, **kwds):
        if (not args) and (not kwds):
            if not hasattr(cls, "_empty"):
                setattr(cls, "_empty", super().__new__(cls))
            return cls._empty
        return super().__new__(cls, *args, **kwds)

    @classmethod
    def partition_kwds(cls, **kwds):
        return partition_mapping(kwds, cls.field_names)



    full_names: bool = True

    __call__ = replace


FmtOpts.partition_kwds(x=1, full_names=False, y=2)



True